In [313]:
import numpy as np
import pandas as pd
import itertools
import math
from xgboost.sklearn import XGBClassifier
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
%matplotlib inline

In [273]:
df = pd.read_csv('train.csv', header = 0)
prom_49 = pd.read_csv('promotion_schedule.csv', header = 0)
price_49 = pd.read_csv('price_49.csv', header = 0)
print(df.shape)

(76502, 5)


In [274]:
y_train = np.ones(df.shape[0])
df['target'] = y_train

In [275]:
i = [n for n in range(0, 2000)]
j = [n for n in range(0, 40)]
t = [n for n in range(0,49)]

In [283]:
%%time
X_train = pd.DataFrame(list(itertools.product(i, j, t)), columns = ['i', 'j', 't'])

Wall time: 3.03 s


In [284]:
price = df.drop_duplicates(subset = ['j','t'])
price.drop(['i', 'target'], axis = 1, inplace = True)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [285]:
X_train.head()

,i,j,t
0,0,0,0
1,0,0,1
2,0,0,2
3,0,0,3
4,0,0,4


In [286]:
X_train = pd.merge(X_train, price, how='left', on = ['j', 't'])
X_train.shape

(3920000, 5)

In [287]:
X_train = pd.merge(X_train, df[['i', 'j', 't', 'target']], how = 'left', on = ['i', 'j', 't'])
X_train.shape

(3920000, 6)

In [288]:
X_train['target'].fillna(0, inplace = True)

In [291]:
%%time
index = X_train['price'].index[X_train['price'].apply(np.isnan)]
for i in index:
    j = int(X_train.iloc[i]['j'])
    t = int(X_train.iloc[i]['t'])
    pr = price.loc[(price['j'] == j) & (price['t'] == t-1)]
    if pr['advertised'].iloc[0] == 0: 
        X_train['price'].iloc[i] = pr['price'].iloc[0]
    else: 
        p = price['price'].loc[(price['j'] == j) & (price['t'] == t-2)]
        X_train['price'].iloc[i] = p.iloc[0]

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


Wall time: 7min 56s


In [295]:
X_train['advertised'].fillna(0, inplace = True)

In [298]:
y_train = X_train['target']
X_train.drop('target', axis= 1, inplace = True)

In [299]:
%%time
logit = LogisticRegressionCV(cv = 3).fit(X_train.drop('t', axis = 1), y_train)

Wall time: 8min 48s


In [302]:
print (logit.scores_)

{1.0: array([[ 0.98048219,  0.98048219,  0.98048219,  0.98048219,  0.98048219,
         0.98048219,  0.98048219,  0.98048219,  0.98048219,  0.98048219],
       [ 0.98048219,  0.98048219,  0.98048219,  0.98048219,  0.98048219,
         0.98048219,  0.98048219,  0.98048219,  0.98048219,  0.98048219],
       [ 0.98048469,  0.98048469,  0.98048469,  0.98048469,  0.98048469,
         0.98048469,  0.98048469,  0.98048469,  0.98048469,  0.98048469],
       [ 0.98048469,  0.98048469,  0.98048469,  0.98048469,  0.98048469,
         0.98048469,  0.98048469,  0.98048469,  0.98048469,  0.98048469],
       [ 0.98048469,  0.98048469,  0.98048469,  0.98048469,  0.98048469,
         0.98048469,  0.98048469,  0.98048469,  0.98048469,  0.98048469],
       [ 0.98048469,  0.98048469,  0.98048469,  0.98048469,  0.98048469,
         0.98048469,  0.98048469,  0.98048469,  0.98048469,  0.98048469],
       [ 0.98048469,  0.98048469,  0.98048469,  0.98048469,  0.98048469,
         0.98048469,  0.98048469,  0.98

In [ ]:
%%time
i = [n for n in range(0, 2000)]
j = [n for n in range(0, 40)]
X_test = pd.DataFrame(list(itertools.product(i, j)), columns = ['i', 'j'])

In [ ]:
y_test = logit.predict_proba(X_test)[:, 1]
y_test = pd.DataFrame(y_test.reshape(len(y_test), -1), columns = ['target'])

In [ ]:
X_test = pd.concat([X_test, y_test], axis = 1)
X_test.drop('discount', axis = 1, inplace = True)

In [307]:
y_train.loc[y_train == 0].shape

(3843498,)

In [311]:
X_test.head()

,i,j,price,advertised,target
0,0,0,1.717944,0,0.999805
1,1,0,1.717944,0,0.999805
2,2,0,1.717944,0,0.999805
3,3,0,1.717944,0,0.999805
4,4,0,1.717944,0,0.999805


In [310]:
X_train.head()

,i,j,t,price,advertised
0,0,0,0,1.717944,0.0
1,0,0,1,1.717944,0.0
2,0,0,2,1.717944,0.0
3,0,0,3,1.717944,0.0
4,0,0,4,1.717944,0.0
